In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, log_loss
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
import numpy as np
import joblib

In [ ]:
# Load dataset
df = pd.read_csv(r"C:\Users\syedb\onedrive\Documents\Bankruptcu_API\data.csv")
df.columns = df.columns.str.strip()  # Remove any whitespace from column names

In [6]:
# Prepare features and target
X = df.drop(columns=["Bankrupt?"])
y = df["Bankrupt?"]

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Train logistic regression model
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [10]:
# Save model and scaler
joblib.dump(model, "model.joblib")
joblib.dump(scaler, "scaler.joblib")

# Predictions for evaluation
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# Evaluation
f1 = f1_score(y_test, y_pred)
logloss = log_loss(y_test, y_proba)
report = classification_report(y_test, y_pred)

# Output evaluation results
print("F1 Score:", f1)
print("Log Loss:", logloss)
print("Classification Report:\n", report)

F1 Score: 0.3025210084033613
Log Loss: 0.3809691963660741
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.88      0.93      1320
           1       0.19      0.82      0.30        44

    accuracy                           0.88      1364
   macro avg       0.59      0.85      0.62      1364
weighted avg       0.97      0.88      0.91      1364



In [11]:
# --- FastAPI Part ---

# Define input schema using Pydantic
class InputData(BaseModel):
    features: List[float]

# Define output schema
class Prediction(BaseModel):
    probability: float
    prediction: int

In [12]:
# Initialize FastAPI app
app = FastAPI()

# Load saved model and scaler
model = joblib.load("model.joblib")
scaler = joblib.load("scaler.joblib")

@app.post("/predict", response_model=Prediction)
def predict(data: InputData):
    X_input = np.array(data.features).reshape(1, -1)
    X_scaled = scaler.transform(X_input)
    probability = model.predict_proba(X_scaled)[0][1]
    prediction = model.predict(X_scaled)[0]
    return Prediction(probability=round(probability, 4), prediction=int(prediction))

# Run the API (optional - for local testing)
# if __name__ == "__main__":
#     uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)